In [ ]:
#packages
import numpy as np
import pymc as pm
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt;
from scipy import stats


data from https://www.kaggle.com/datasets/arashnic/fitbit/data
The relation between the "TotalSteps" and "Calories
we pick 1000 of them

In [ ]:
data = pd.read_csv('dailyActivity_merged.csv')
data_1 = data.head(1000)


In [ ]:

n = len(data_1.axes[0])
p = len(data_1.axes[1])
X,y=np.zeros((n,p)),np.ones((n,1))

with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()

In [ ]:
n = len(data_1.axes[0])
p = len(data_1.axes[1])
X,y=np.zeros((n,p)),np.ones((n,1))
Psi=np.eye(p); a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)
y=stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)

    
with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    Sigma = pm.Deterministic('Sigma', L.dot(L.T)) 
    mu = pm.MvNormal('mu', mu=np.array(0), cov=np.eye(p), shape=p); 
    y = pm.MvNormal('y', mu=mu, cov=Sigma, shape=(n,1), observed=y) 
   
with MNV_LKJ:    
    idata = pm.sample()